In [53]:
import sys, os, json, time
from collections import Counter

In [65]:
import pandas as pd
import numpy as np

In [4]:
with open('secret.conf', 'r') as f:
    for line in f:
        line = line.strip().split(' ')
        if line[0] == 'export':
            equal_sign_position = line[1].find('=')
            k = line[1][:equal_sign_position]
            v = line[1][equal_sign_position+1:].strip('"')
            os.environ[k] = v
            

In [5]:
from ms_academic import MicrosoftAcademic

In [6]:
api = MicrosoftAcademic()

In [20]:
api.attributes = 'Id,AA.AuId,AA.AuN,AA.AfN,AA.AfId'

In [21]:
r = api.interpret_and_evaluate('tamara munzner')

In [24]:
r

[{u'AA': [{u'AfId': 141945490,
    u'AfN': u'university of british columbia',
    u'AuId': 544170546,
    u'AuN': u'tamara munzner'},
   {u'AfId': 66946132,
    u'AfN': u'university of maryland college park',
    u'AuId': 2289381742,
    u'AuN': u'francois guimbretiere'},
   {u'AfId': 1324840837,
    u'AfN': u'hewlett packard',
    u'AuId': 80569443,
    u'AuN': u'serdar tasiran'},
   {u'AfId': 1324840837,
    u'AfN': u'hewlett packard',
    u'AuId': 2305686494,
    u'AuN': u'li zhang'},
   {u'AfId': 1324840837,
    u'AfN': u'hewlett packard',
    u'AuId': 2155061604,
    u'AuN': u'yunhong zhou'}],
  u'Id': 2074494141,
  u'logprob': -17.716},
 {u'AA': [{u'AfId': 141945490,
    u'AfN': u'university of british columbia',
    u'AuId': 544170546,
    u'AuN': u'tamara munzner'}],
  u'Id': 2142493242,
  u'logprob': -17.981},
 {u'AA': [{u'AfId': 141945490,
    u'AfN': u'university of british columbia',
    u'AuId': 2078060999,
    u'AuN': u'michael sedlmair'},
   {u'AfId': 223532165,
    u'Af

In [26]:
for item in r:
    for aa in item['AA']:
        if aa['AuN'] == 'tamara munzner':
            print("{}: {}".format(aa['AfId'], aa['AfN']))

141945490: university of british columbia
141945490: university of british columbia
141945490: university of british columbia
97018004: stanford university
141945490: university of british columbia
141945490: university of british columbia
141945490: university of british columbia
141945490: university of british columbia
97018004: stanford university
141945490: university of british columbia


In [29]:
with open('coauthorship.json', 'r') as f:
    j = json.load(f)

In [35]:
x = j['nodes'][0]
x['author_name']

u'Daniel Jackson'

In [57]:
start = time.time()
authors = {}
none_found = []
i = 0
for node in j['nodes']:
    author_name = node['author_name'].lower()
    r = api.interpret_and_evaluate(author_name)
    if author_name not in authors:
        authors[author_name] = {'author_id': Counter(), 'affil_name': Counter(), 'affil_id': Counter()}
    found = False
    for item in r:
        for aa in item['AA']:
            if aa['AuN'] == author_name:
                found = True
                author_id = aa.get('AuId')
                affil_name = aa.get('AfN')
                affil_id = aa.get('AfId')
                authors[author_name]['author_id'][author_id] += 1
                authors[author_name]['affil_name'][affil_name] += 1
                authors[author_name]['affil_id'][affil_id] += 1
    if not found:
        none_found.append(author_name)
    i += 1
    if i % 10 == 0:
        print("{} queries in {:.1f} seconds. sleeping 10 second".format(i, time.time()-start))
        time.sleep(10)

10 queries in 5.5 seconds. sleeping 1 second
20 queries in 19.9 seconds. sleeping 1 second
30 queries in 35.5 seconds. sleeping 1 second
40 queries in 51.2 seconds. sleeping 1 second
50 queries in 69.5 seconds. sleeping 1 second
60 queries in 84.4 seconds. sleeping 1 second
70 queries in 101.0 seconds. sleeping 1 second
80 queries in 117.7 seconds. sleeping 1 second
90 queries in 133.6 seconds. sleeping 1 second
100 queries in 148.8 seconds. sleeping 1 second
110 queries in 166.1 seconds. sleeping 1 second
120 queries in 182.2 seconds. sleeping 1 second
130 queries in 196.9 seconds. sleeping 1 second
140 queries in 213.6 seconds. sleeping 1 second
150 queries in 228.3 seconds. sleeping 1 second
160 queries in 243.1 seconds. sleeping 1 second
170 queries in 258.3 seconds. sleeping 1 second
180 queries in 273.4 seconds. sleeping 1 second
190 queries in 288.2 seconds. sleeping 1 second
200 queries in 309.3 seconds. sleeping 1 second
210 queries in 325.2 seconds. sleeping 1 second
220 quer

KeyError: 'expr'

In [58]:
len(authors)

271

In [60]:
len(none_found)

99

In [62]:
import cPickle as pickle
with open('author_affil_mag-INCOMPLETE.pickle', 'wb') as outf:
    pickle.dump(authors, outf)

In [68]:
authors['']['affil_name'].most_common()

KeyError: 'tamara munzner'

In [91]:
rows = []
for node in j['nodes']:
    cl_bottom = node['cl_bottom']
    cl_top = node['cl_top']
    author_name = node['author_name']
    cl_depth = len(cl_bottom.split(':'))
    if author_name.lower() in authors:
        affil_name = authors[author_name.lower()]['affil_name']
        if affil_name:
            affil_name = affil_name.most_common()[0][0]
        else:
            affil_name = ''
        rows.append( (author_name, cl_bottom, cl_top, cl_depth, affil_name) )
df = pd.DataFrame(rows, columns='author_name cl_bottom cl_top cl_depth affil_name'.split())

In [93]:
df.sort_values(['cl_depth', 'cl_bottom'], ascending=False)

,author_name,cl_bottom,cl_top,cl_depth,affil_name
24,Takafumi Saito,2:1:2:2,2,4,yamagata university
136,Jun Sese,2:1:2:2,2,4,university of tokyo
177,Takayuki Itoh,2:1:2:2,2,4,ibm
40,Michael Ogawa,2:1:2:1,2,4,university of california davis
130,Chris Muelder,2:1:2:1,2,4,university of california davis
142,Kwan-Liu Ma,2:1:2:1,2,4,
175,Diane Felmlee,2:1:2:1,2,4,
215,Robert Faris,2:1:2:1,2,4,university of california davis
138,Fr\'ed\'eric Lalanne,2:1:1:5,2,4,
22,Aaron Barsky,2:1:1:4,2,4,university of british columbia


In [94]:
df[df.author_name.isin(['Aaron Barsky', 'Tamara Munzner'])]

,author_name,cl_bottom,cl_top,cl_depth,affil_name
22,Aaron Barsky,2:1:1:4,2,4,university of british columbia


In [85]:
affil_name.most_common(1)[0][0]

u'massachusetts institute of technology'

In [70]:
node

{u'author_name': u'Daniel Jackson',
 u'cl_bottom': u'27',
 u'cl_top': u'27',
 u'component': 0,
 u'flow': 0.00138504,
 u'id': 1,
 u'papers': [{u'keywords': [u'paper_type:application',
    u'group_type:vertex-based',
    u'graph_vis:matrix',
    u'group_overlap:disjoint',
    u'group_structure:hierarchical',
    u'edge-group_vis:none',
    u'group_vis:juxtaposed',
    u'visual_attr:!',
    u'juxtaposed_vis:attached',
    u'superimposed_vis:!',
    u'embedded_vis:!',
    u'implicit_vis:1D_layout',
    u'graph:directed',
    u'group_origin:generic',
    u'evaluation:case_study',
    u'application:software_engineering',
    u'publication_channel:oopsla',
    u'aggregation'],
   u'title': u'Using Dependency Models to Manage Complex Software Architecture',
   u'year': u'2005'}]}